https://zys-notes.blogspot.com/2020/01/api.html

In [1]:

from IPython.display import display, clear_output
from urllib.request import urlopen
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import requests
import sched
import time
import json
import twstock
import FinMind
from FinMind.data import DataLoader
from FinMind import plotting
#Data Source
import yfinance as yf

#Data viz
import plotly.graph_objs as go


import pyodbc

import schedule
import sys



C:\Anaconda3\envs\Py37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Anaconda3\envs\Py37\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Anaconda3\envs\Py37\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
def stock_realtime_fn(targets):
    #today date 
    time = datetime.datetime.now().date()

    #stock_list
    stock_list = '|'.join('tse_{}.tw'.format(target) for target in targets) 

    #　query data
    query_url = "http://mis.twse.com.tw/stock/api/getStockInfo.jsp?ex_ch="+ stock_list
    data = json.loads(urlopen(query_url).read())
#
    # get data
    columns = ['t','c','n','z','tv','v','o','h','l','y']
    df = pd.DataFrame(data['msgArray'], columns=columns)
    df0 = pd.DataFrame(data['msgArray'], columns=['f','g','b','a'])
    df1 = pd.DataFrame({
        'bid_volumn':df0['f'].str.split('_', expand=True)[0],
        'ask_volumn':df0['g'].str.split('_', expand=True)[0],     
        'bid_price':df0['b'].str.split('_', expand=True)[0],
        'ask_price':df0['a'].str.split('_', expand=True)[0],
        'bid_volumn1':df0['f'].str.split('_', expand=True)[1],
        'ask_volumn1':df0['g'].str.split('_', expand=True)[1],     
        'bid_price1':df0['b'].str.split('_', expand=True)[1],
        'ask_price1':df0['a'].str.split('_', expand=True)[1]  
                 })
    df.columns = ['curr_time','tickers','namec','current_price','current_volumn','acc_volumn','s_open','hight','low','y_close']
    df = pd.concat([df, df1], axis=1)
    df = df[df['current_price']!='-']
    for i in ['current_price','s_open','hight','low','y_close','bid_price','ask_price','bid_price1','ask_price1']:
        df[i] = df[i].apply(lambda x: round(float(x),2))
    for i in ['current_volumn','acc_volumn', 'bid_volumn','ask_volumn', 'bid_volumn1','ask_volumn1']:
        df[i] = df[i].apply(lambda x: int(x))
    # 增加時間
    df['date'] = time
    # 紀錄更新時間
    time0 = datetime.datetime.now()
    #print(df)
    return df
#stock_realtime_fn(targets = ['2454','2330']).columns

In [3]:
def get_stock_exec(data_table_name,st_list= ['2454','2330']):
    data = stock_realtime_fn(st_list)
    # connection    
    server = 'PC-20191205RA' 
    database = 'stock_historical_all' 
    username = 'sa' 
    password = '123' 
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    # create table
    index = True
    if index == True :
        index = False
        sql = "IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='"+data_table_name +"' AND xtype='U')"+'CREATE TABLE '+data_table_name+''' ( 
                    date nvarchar(16),
                    curr_time nvarchar(10), 
                    tickers nvarchar(10), 
                    name nvarchar(10), 
                    current_price NUMERIC(10,2), 
                    current_volumn int,
                    bid_volumn int,
                    ask_volumn int, 
                    bid_price NUMERIC(10,2), 
                    ask_price NUMERIC(10,2), 
                    bid_volumn1 int, 
                    ask_volumn1 int,
                    bid_price1 NUMERIC(10,2), 
                    ask_price1 NUMERIC(10,2),
                    acc_volumn int, 
                    s_open NUMERIC(10,2), 
                    hight NUMERIC(10,2), 
                    low NUMERIC(10,2), 
                    y_close NUMERIC(10,2)
                    )
        '''
        # excute1
        cursor = cnxn.cursor()
        cursor.execute(sql)
        cnxn.commit()
        
    cursor = cnxn.cursor()
    # Insert Dataframe into SQL Server:
    sql = "INSERT INTO " +data_table_name +'''
     (date, curr_time, tickers, name, current_price, current_volumn, 
     bid_volumn, ask_volumn, bid_price, ask_price, 
     acc_volumn, s_open, hight, low, y_close) values(?,?,?,?,?,?
     ,?,?,?,?
     ,?,?,?,?,?) 
    '''
     
    for index, row in data.iterrows():
         cursor.execute(sql,
                        str(row.date), row.curr_time, row.tickers,row.namec, row.current_price, row.current_volumn, 
                        row.bid_volumn, row.ask_volumn , row.bid_price, row.ask_price,
                        row.acc_volumn, row.s_open,  row.hight,  row.low,  
                        row.y_close)        

    cnxn.commit()
    cursor.close()
    
def exit():
    print('Now the system will exit ') 
    sys.exit()



In [9]:
schedule.every(2).seconds.do(get_stock_exec,data_table_name='stock_realtime_v2',st_list= ['2454','2330'])   
schedule.every().day.at('13:31:00').do(exit)

while True:
    schedule.run_pending()

更新時間:2022-02-24 08:59:37.194631
更新時間:2022-02-24 08:59:49.545568
更新時間:2022-02-24 09:00:01.854651
更新時間:2022-02-24 09:00:04.002856
更新時間:2022-02-24 09:00:16.383775
更新時間:2022-02-24 09:00:18.537978
更新時間:2022-02-24 09:00:30.862015
更新時間:2022-02-24 09:00:43.132222
更新時間:2022-02-24 09:00:55.508134
更新時間:2022-02-24 09:00:57.642425
更新時間:2022-02-24 09:00:59.775720
更新時間:2022-02-24 09:01:12.150619
更新時間:2022-02-24 09:01:14.283914
更新時間:2022-02-24 09:01:16.431170
更新時間:2022-02-24 09:01:18.594384
更新時間:2022-02-24 09:01:20.740643
更新時間:2022-02-24 09:01:22.905851
更新時間:2022-02-24 09:01:25.040142
更新時間:2022-02-24 09:01:27.175432
更新時間:2022-02-24 09:01:29.326677
更新時間:2022-02-24 09:01:31.461965
更新時間:2022-02-24 09:01:33.599250
更新時間:2022-02-24 09:01:45.920293
更新時間:2022-02-24 09:01:48.056579
更新時間:2022-02-24 09:01:50.194861
更新時間:2022-02-24 09:01:52.383007
更新時間:2022-02-24 09:01:54.520290
更新時間:2022-02-24 09:01:56.657575
更新時間:2022-02-24 09:02:09.019090
更新時間:2022-02-24 09:02:11.176318
更新時間:2022-02-24 09:02:13.318627
更新時間:202

更新時間:2022-02-24 09:11:33.114250
更新時間:2022-02-24 09:11:35.265496
更新時間:2022-02-24 09:11:37.399788
更新時間:2022-02-24 09:11:39.534080
更新時間:2022-02-24 09:11:41.731230
更新時間:2022-02-24 09:11:43.869510
更新時間:2022-02-24 09:11:46.021726
更新時間:2022-02-24 09:11:48.160010
更新時間:2022-02-24 09:11:50.297289
更新時間:2022-02-24 09:11:52.449533
更新時間:2022-02-24 09:11:54.586853
更新時間:2022-02-24 09:11:56.728119
更新時間:2022-02-24 09:11:58.881333
更新時間:2022-02-24 09:12:01.064492
更新時間:2022-02-24 09:12:03.305525
更新時間:2022-02-24 09:12:05.526583
更新時間:2022-02-24 09:12:07.685809
更新時間:2022-02-24 09:12:09.843038
更新時間:2022-02-24 09:12:11.988273
更新時間:2022-02-24 09:12:14.133581
更新時間:2022-02-24 09:12:16.301763
更新時間:2022-02-24 09:12:18.479909
更新時間:2022-02-24 09:12:20.626196
更新時間:2022-02-24 09:12:22.816310
更新時間:2022-02-24 09:12:24.996479
更新時間:2022-02-24 09:12:27.155731
更新時間:2022-02-24 09:12:29.401698
更新時間:2022-02-24 09:12:31.567929
更新時間:2022-02-24 09:12:43.944844
更新時間:2022-02-24 09:12:46.136933
更新時間:2022-02-24 09:12:48.278233
更新時間:202

更新時間:2022-02-24 09:21:19.583203
更新時間:2022-02-24 09:21:21.720486
更新時間:2022-02-24 09:21:23.879720
更新時間:2022-02-24 09:21:36.255582
更新時間:2022-02-24 09:21:38.391871
更新時間:2022-02-24 09:21:40.549099
更新時間:2022-02-24 09:21:42.709318
更新時間:2022-02-24 09:21:44.863556
更新時間:2022-02-24 09:21:47.019837
更新時間:2022-02-24 09:21:49.181010
更新時間:2022-02-24 09:21:51.318294
更新時間:2022-02-24 09:21:53.473574
更新時間:2022-02-24 09:21:55.658705
更新時間:2022-02-24 09:21:57.860823
更新時間:2022-02-24 09:22:00.021014
更新時間:2022-02-24 09:22:02.176297
更新時間:2022-02-24 09:22:04.349487
更新時間:2022-02-24 09:22:06.485724
更新時間:2022-02-24 09:22:08.639989
更新時間:2022-02-24 09:22:10.792207
更新時間:2022-02-24 09:22:12.925526
更新時間:2022-02-24 09:22:15.077741
更新時間:2022-02-24 09:22:17.214026
更新時間:2022-02-24 09:22:19.354331
更新時間:2022-02-24 09:22:21.544445
更新時間:2022-02-24 09:22:23.685717
更新時間:2022-02-24 09:22:25.842947
更新時間:2022-02-24 09:22:27.986242
更新時間:2022-02-24 09:22:30.123498
更新時間:2022-02-24 09:22:32.275768
更新時間:2022-02-24 09:22:34.423997
更新時間:202

更新時間:2022-02-24 09:31:04.142298
更新時間:2022-02-24 09:31:06.282601
更新時間:2022-02-24 09:31:08.439802
更新時間:2022-02-24 09:31:10.583092
更新時間:2022-02-24 09:31:12.732347
更新時間:2022-02-24 09:31:14.885588
更新時間:2022-02-24 09:31:17.018883
更新時間:2022-02-24 09:31:19.155177
更新時間:2022-02-24 09:31:21.311374
更新時間:2022-02-24 09:31:23.450685
更新時間:2022-02-24 09:31:25.589957
更新時間:2022-02-24 09:31:27.743169
更新時間:2022-02-24 09:31:29.896433
更新時間:2022-02-24 09:31:32.050649
更新時間:2022-02-24 09:31:34.185936
更新時間:2022-02-24 09:31:36.321226
更新時間:2022-02-24 09:31:38.476487
更新時間:2022-02-24 09:31:40.613771
更新時間:2022-02-24 09:31:42.769019
更新時間:2022-02-24 09:31:44.927233
更新時間:2022-02-24 09:31:47.063492
更新時間:2022-02-24 09:31:49.198808
更新時間:2022-02-24 09:31:51.361015
更新時間:2022-02-24 09:31:53.500278
更新時間:2022-02-24 09:31:55.639555
更新時間:2022-02-24 09:31:57.795820
更新時間:2022-02-24 09:31:59.932099
更新時間:2022-02-24 09:32:02.110280
更新時間:2022-02-24 09:32:04.268472
更新時間:2022-02-24 09:32:06.405756
更新時間:2022-02-24 09:32:08.562986
更新時間:202

更新時間:2022-02-24 09:40:48.495296
更新時間:2022-02-24 09:40:50.629614
更新時間:2022-02-24 09:40:52.790841
更新時間:2022-02-24 09:40:54.928138
更新時間:2022-02-24 09:40:57.084350
更新時間:2022-02-24 09:40:59.218614
更新時間:2022-02-24 09:41:01.389855
更新時間:2022-02-24 09:41:03.546038
更新時間:2022-02-24 09:41:05.718279
更新時間:2022-02-24 09:41:07.871472
更新時間:2022-02-24 09:41:10.007777
更新時間:2022-02-24 09:41:12.142096
更新時間:2022-02-24 09:41:14.349170
更新時間:2022-02-24 09:41:16.482471
更新時間:2022-02-24 09:41:18.620716
更新時間:2022-02-24 09:41:20.781963
更新時間:2022-02-24 09:41:22.919221
更新時間:2022-02-24 09:41:25.083461
更新時間:2022-02-24 09:41:27.218747
更新時間:2022-02-24 09:41:29.352043
更新時間:2022-02-24 09:41:31.526226
更新時間:2022-02-24 09:41:33.664508
更新時間:2022-02-24 09:41:35.811764
更新時間:2022-02-24 09:41:38.017856
更新時間:2022-02-24 09:41:40.156115
更新時間:2022-02-24 09:41:42.295422
更新時間:2022-02-24 09:41:44.453620
更新時間:2022-02-24 09:41:46.609854
更新時間:2022-02-24 09:41:58.921949
更新時間:2022-02-24 09:42:01.062198
更新時間:2022-02-24 09:42:03.220423
更新時間:202

...

更新時間:2022-02-24 12:50:54.842762
更新時間:2022-02-24 12:50:56.983876
更新時間:2022-02-24 12:50:59.122021
更新時間:2022-02-24 12:51:01.308066
更新時間:2022-02-24 12:51:03.476103
更新時間:2022-02-24 12:51:05.633199
更新時間:2022-02-24 12:51:07.780349
更新時間:2022-02-24 12:51:09.918469
更新時間:2022-02-24 12:51:12.076561
更新時間:2022-02-24 12:51:14.215707
更新時間:2022-02-24 12:51:16.360862
更新時間:2022-02-24 12:51:18.520922
更新時間:2022-02-24 12:51:20.702951
更新時間:2022-02-24 12:51:22.857055
更新時間:2022-02-24 12:51:24.999192
更新時間:2022-02-24 12:51:27.133349
更新時間:2022-02-24 12:51:29.288451
更新時間:2022-02-24 12:51:31.425599
更新時間:2022-02-24 12:51:33.578735
更新時間:2022-02-24 12:51:35.716852
更新時間:2022-02-24 12:51:37.866967
更新時間:2022-02-24 12:51:40.023069
更新時間:2022-02-24 12:51:42.162239
更新時間:2022-02-24 12:51:44.309335
更新時間:2022-02-24 12:51:46.501365
更新時間:2022-02-24 12:51:48.637488
更新時間:2022-02-24 12:51:50.791591
更新時間:2022-02-24 12:51:52.926773
更新時間:2022-02-24 12:51:55.061904
更新時間:2022-02-24 12:51:57.234953
更新時間:2022-02-24 12:51:59.368114
更新時間:202

更新時間:2022-02-24 13:02:23.209536
更新時間:2022-02-24 13:02:25.347679
更新時間:2022-02-24 13:02:27.589543
更新時間:2022-02-24 13:02:29.726696
更新時間:2022-02-24 13:02:31.878804
更新時間:2022-02-24 13:02:34.013957
更新時間:2022-02-24 13:02:36.146124
更新時間:2022-02-24 13:02:38.299227
更新時間:2022-02-24 13:02:40.435380
更新時間:2022-02-24 13:02:42.587490
更新時間:2022-02-24 13:02:44.722650
更新時間:2022-02-24 13:02:46.857801
更新時間:2022-02-24 13:02:49.010935
更新時間:2022-02-24 13:02:51.145068
更新時間:2022-02-24 13:02:53.279227
更新時間:2022-02-24 13:02:55.431368
更新時間:2022-02-24 13:02:57.586438
更新時間:2022-02-24 13:02:59.740541
更新時間:2022-02-24 13:03:01.883675
更新時間:2022-02-24 13:03:04.018832
更新時間:2022-02-24 13:03:06.171934
更新時間:2022-02-24 13:03:08.310086
更新時間:2022-02-24 13:03:10.487124
更新時間:2022-02-24 13:03:12.643223
更新時間:2022-02-24 13:03:14.780370
更新時間:2022-02-24 13:03:16.921511
更新時間:2022-02-24 13:03:19.080600
更新時間:2022-02-24 13:03:21.219747
更新時間:2022-02-24 13:03:23.361881
更新時間:2022-02-24 13:03:25.520973
更新時間:2022-02-24 13:03:27.657127
更新時間:202

更新時間:2022-02-24 13:12:31.667688
更新時間:2022-02-24 13:12:33.806604
更新時間:2022-02-24 13:12:35.974072
更新時間:2022-02-24 13:12:38.110173
更新時間:2022-02-24 13:12:40.276992
更新時間:2022-02-24 13:12:42.458444
更新時間:2022-02-24 13:12:44.648928
更新時間:2022-02-24 13:12:46.816799
更新時間:2022-02-24 13:12:49.077104
更新時間:2022-02-24 13:12:51.221721
更新時間:2022-02-24 13:12:53.395097
更新時間:2022-02-24 13:12:55.578352
更新時間:2022-02-24 13:12:57.741620
更新時間:2022-02-24 13:12:59.906728
更新時間:2022-02-24 13:13:02.062501
更新時間:2022-02-24 13:13:05.337222
更新時間:2022-02-24 13:13:07.496061
更新時間:2022-02-24 13:13:09.686164
更新時間:2022-02-24 13:13:11.837938
更新時間:2022-02-24 13:13:24.243157
更新時間:2022-02-24 13:13:36.633213
更新時間:2022-02-24 13:13:38.805193
更新時間:2022-02-24 13:13:40.964655
更新時間:2022-02-24 13:13:43.127151
更新時間:2022-02-24 13:13:45.334015
更新時間:2022-02-24 13:13:47.523312
更新時間:2022-02-24 13:13:50.684556
更新時間:2022-02-24 13:13:52.870614
更新時間:2022-02-24 13:13:55.021283
更新時間:2022-02-24 13:13:57.156049
更新時間:2022-02-24 13:13:59.315511
更新時間:202

更新時間:2022-02-24 13:23:27.819004
更新時間:2022-02-24 13:23:29.981207
更新時間:2022-02-24 13:23:32.141787
更新時間:2022-02-24 13:23:34.326528
更新時間:2022-02-24 13:23:36.502849
更新時間:2022-02-24 13:23:38.677640
更新時間:2022-02-24 13:23:40.865299
更新時間:2022-02-24 13:23:43.019569
更新時間:2022-02-24 13:23:55.423412
更新時間:2022-02-24 13:23:57.575560
更新時間:2022-02-24 13:23:59.760074
更新時間:2022-02-24 13:24:01.916225
更新時間:2022-02-24 13:24:04.236409
更新時間:2022-02-24 13:24:07.432436
更新時間:2022-02-24 13:24:09.601103
更新時間:2022-02-24 13:24:11.785367
更新時間:2022-02-24 13:24:13.936977
更新時間:2022-02-24 13:24:16.120428
更新時間:2022-02-24 13:24:18.284345
更新時間:2022-02-24 13:24:20.439505
更新時間:2022-02-24 13:24:22.600405
更新時間:2022-02-24 13:24:24.753358
更新時間:2022-02-24 13:24:26.947157
更新時間:2022-02-24 13:24:29.160464
更新時間:2022-02-24 13:24:31.319807
更新時間:2022-02-24 13:24:33.507467
更新時間:2022-02-24 13:24:35.649597
更新時間:2022-02-24 13:24:37.822148
更新時間:2022-02-24 13:24:40.061793
更新時間:2022-02-24 13:24:42.228406
更新時間:2022-02-24 13:24:44.384287
更新時間:202

SystemExit: 

In [8]:
schedule.clear()